# Improved Formant-focused Training (V2)

Training improved version of Formant-focused model:
- Increased formant branch: 3→128→64→32 (with residual connections)
- Increased other features: (n-3)→512→256→128
- Multi-head attention for formants (instead of simple softmax)
- Improved fusion: 32+128→256→128→64→2
- Better training: 80 epochs, warmup, cosine annealing


In [1]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

PROJECT_ROOT = Path('/Volumes/SSanDisk/SpeechRec-German/without_context_windows')
sys.path.insert(0, str(PROJECT_ROOT))

from models.specialized.formant_focused_v2 import FormantFocusedModel_V2
from utils.training_utils import train_model, evaluate_model, WarmupCosineScheduler, LabelSmoothingCrossEntropy
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_loader = dataloaders['feature']['train']
val_loader = dataloaders['feature']['val']
test_loader = dataloaders['feature']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models' / 'improved_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)


Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model: Improved Formant-focused V2


In [2]:
model = FormantFocusedModel_V2(n_features=len(feature_cols), num_classes=2, dropout=0.2).to(device)

# Set formant indices
model.set_formant_indices(feature_cols, ['formant_f1', 'formant_f2', 'formant_f3'])

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model: {model.get_config()['model_type']}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Loss function with label smoothing
criterion = LabelSmoothingCrossEntropy(smoothing=0.1, weight=class_weights)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)

# Learning rate scheduler with warmup and cosine annealing
num_epochs = 80
warmup_epochs = 5
scheduler = WarmupCosineScheduler(optimizer, warmup_epochs=warmup_epochs, total_epochs=num_epochs, min_lr=1e-6)

save_dir = OUTPUT_DIR / 'formant_focused_v2'
save_dir.mkdir(parents=True, exist_ok=True)

print(f"\nTraining configuration:")
print(f"- Epochs: {num_epochs}")
print(f"- Warmup epochs: {warmup_epochs}")
print(f"- Initial LR: {optimizer.param_groups[0]['lr']}")
print(f"- Label smoothing: 0.1")
print(f"- Gradient clipping: 1.0")
print(f"- Early stopping patience: 15")


Model: FormantFocusedModel_V2
Total parameters: 327,890
Trainable parameters: 327,890

Training configuration:
- Epochs: 80
- Warmup epochs: 5
- Initial LR: 0.0005
- Label smoothing: 0.1
- Gradient clipping: 1.0
- Early stopping patience: 15


In [3]:
history, best_epoch = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    device, num_epochs=num_epochs, save_dir=save_dir, model_name='formant_focused_v2', 
    early_stopping_patience=15, max_grad_norm=1.0
)

checkpoint = torch.load(save_dir / 'best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
test_metrics, _, _, _ = evaluate_model(model, test_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics, f, indent=2)

print(f"\n{'='*60}")
print(f"Final Test Results:")
print(f"{'='*60}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"F1-score: {test_metrics['f1']:.4f}")
print(f"ROC-AUC: {test_metrics['roc_auc']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"Best epoch: {best_epoch}")



Epoch 1/80
--------------------------------------------------


Train Loss: 0.4126, Train Acc: 0.8875
Val Loss: 0.3463, Val Acc: 0.9198
Val F1: 0.9211, Val ROC-AUC: 0.9762
Learning Rate: 0.000100
✓ New best model saved! (F1: 0.9211)

Epoch 2/80
--------------------------------------------------


Train Loss: 0.3622, Train Acc: 0.9202
Val Loss: 0.3424, Val Acc: 0.9205
Val F1: 0.9219, Val ROC-AUC: 0.9780
Learning Rate: 0.000200
✓ New best model saved! (F1: 0.9219)

Epoch 3/80
--------------------------------------------------


Train Loss: 0.3569, Train Acc: 0.9226
Val Loss: 0.3356, Val Acc: 0.9346
Val F1: 0.9350, Val ROC-AUC: 0.9794
Learning Rate: 0.000300
✓ New best model saved! (F1: 0.9350)

Epoch 4/80
--------------------------------------------------


Train Loss: 0.3516, Train Acc: 0.9266
Val Loss: 0.3368, Val Acc: 0.9254
Val F1: 0.9266, Val ROC-AUC: 0.9799
Learning Rate: 0.000400

Epoch 5/80
--------------------------------------------------


...

Train Loss: 0.2474, Train Acc: 0.9933
Val Loss: 0.3578, Val Acc: 0.9453
Val F1: 0.9450, Val ROC-AUC: 0.9735
Learning Rate: 0.000003

Epoch 78/80
--------------------------------------------------


Train Loss: 0.2468, Train Acc: 0.9930
Val Loss: 0.3475, Val Acc: 0.9445
Val F1: 0.9446, Val ROC-AUC: 0.9712
Learning Rate: 0.000002

Epoch 79/80
--------------------------------------------------


Train Loss: 0.2449, Train Acc: 0.9934
Val Loss: 0.3516, Val Acc: 0.9447
Val F1: 0.9447, Val ROC-AUC: 0.9713
Learning Rate: 0.000001

Epoch 80/80
--------------------------------------------------


Train Loss: 0.2490, Train Acc: 0.9918
Val Loss: 0.3516, Val Acc: 0.9466
Val F1: 0.9465, Val ROC-AUC: 0.9753
Learning Rate: 0.000001
✓ New best model saved! (F1: 0.9465)



Final Test Results:
Accuracy: 0.9413
F1-score: 0.9411
ROC-AUC: 0.9689
Precision: 0.9410
Recall: 0.9413
Best epoch: 80
